In [2]:
import pandas as pd
import math
import numpy as np
from konlpy.tag import Okt
okt=Okt()
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from Korpora import Korpora
corpus = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
     Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



In [5]:
df = pd.read_csv("shopping_comment\바라던유통_바라던_목살_구이용_300g.csv")

In [18]:
df["comment"].values

'한팩당 다섯 슬라이스로 되어있는것같고 6팩 가량 구매했는데 정말 꽝꽝 냉동 및 진공 포장을 한것도 모자라 아이스팩까지 넣어오기에 한여름에 온다해도 상하는 등의 문제가 없을것같습니다.(이쯤되면 여름에도 안녹을듯) 또한 구워먹으니 담백하고 가격도 저렴하여 정말 부담없이 즐길 수 있습니다. 스팸과 같은 가공식품 드시는 분이라면 이걸 드시는게 차라리 건강에는 더 좋을것같습니다 1슬라이스정도면 고기를 구워서 쌈싸먹고 그렇게 후하게먹는 용도가 아닌 반찬용으로도 적당합니다. 냉면등등 사드신다면 고기랑 곁들여먹으면 집이 곧 식당입니다.'

In [19]:
# corpus = corpus.train[0:500].text
corpus = df["comment"].values

In [20]:
pos_tagged = []
for sentence in corpus:
    pos_tagged.append(okt.pos(sentence))

In [21]:
wordlist = []
for sentence in pos_tagged:
    for word in sentence:
        wordlist.append(word)
wordlist

[('한', 'Determiner'),
 ('팩', 'Noun'),
 ('당', 'Suffix'),
 ('다섯', 'Noun'),
 ('슬', 'Noun'),
 ('라이스', 'Noun'),
 ('로', 'Josa'),
 ('되어있는것', 'Verb'),
 ('같고', 'Adjective'),
 ('6', 'Number'),
 ('팩', 'Noun'),
 ('가량', 'Noun'),
 ('구매', 'Noun'),
 ('했는데', 'Verb'),
 ('정말', 'Noun'),
 ('꽝꽝', 'Adverb'),
 ('냉동', 'Noun'),
 ('및', 'Noun'),
 ('진공', 'Noun'),
 ('포장', 'Noun'),
 ('을', 'Josa'),
 ('한', 'Determiner'),
 ('것', 'Noun'),
 ('도', 'Josa'),
 ('모자라', 'Noun'),
 ('아이스팩', 'Noun'),
 ('까지', 'Josa'),
 ('넣어오기에', 'Verb'),
 ('한여름', 'Noun'),
 ('에', 'Josa'),
 ('온', 'Modifier'),
 ('다해', 'Noun'),
 ('도', 'Josa'),
 ('상하', 'Noun'),
 ('는', 'Josa'),
 ('등', 'Noun'),
 ('의', 'Josa'),
 ('문제', 'Noun'),
 ('가', 'Josa'),
 ('없을것', 'Adjective'),
 ('같습니다', 'Adjective'),
 ('.(', 'Punctuation'),
 ('이쯤', 'Noun'),
 ('되면', 'Verb'),
 ('여름', 'Noun'),
 ('에도', 'Josa'),
 ('안', 'VerbPrefix'),
 ('녹을듯', 'Verb'),
 (')', 'Punctuation'),
 ('또한', 'Noun'),
 ('구워', 'Verb'),
 ('먹으니', 'Verb'),
 ('담백하고', 'Adjective'),
 ('가격', 'Noun'),
 ('도', 'Josa'),
 ('저렴하

In [22]:
word2index={}
bow = []
for word in wordlist:
    if word not in word2index.keys():
        word2index[word]=len(word2index)
        bow.append([word, 1])
    else:
        index=word2index.get(word)
        bow[index] = [word, bow[index][1]+1]

In [23]:
wordlist = sorted(bow, key=lambda word: word[1], reverse=True)   

In [24]:
f = open("한국어불용어100.txt", 'rt', encoding='UTF8')
stopwords = f.readlines()
f.close()

In [25]:
stopwords = [i.split('\t')[0].strip() for i in stopwords]

In [30]:
stopwords += ['고기', '정말', '너무', '진짜']

In [31]:
toberemoved = ["Punctuation", "Josa", "Modifier", "Number", "Suffix", "Determiner", "VerbPrefix", "Alpha", "Verb"]
result = []
for word in wordlist:
    if word[0][1] not in toberemoved and word[1] > 3 and word[0][0] not in stopwords and len(word[0][0]) > 1:
        result.append(word)
result

[[('배송', 'Noun'), 56],
 [('좋아요', 'Adjective'), 47],
 [('포장', 'Noun'), 43],
 [('맛있어요', 'Adjective'), 30],
 [('구매', 'Noun'), 27],
 [('가격', 'Noun'), 26],
 [('목살', 'Noun'), 25],
 [('좋습니다', 'Adjective'), 25],
 [('냄새', 'Noun'), 24],
 [('입니다', 'Adjective'), 21],
 [('빠르고', 'Adjective'), 21],
 [('주문', 'Noun'), 21],
 [('항상', 'Noun'), 20],
 [('가성', 'Noun'), 15],
 [('조아요', 'Adjective'), 15],
 [('아주', 'Noun'), 14],
 [('맛있게', 'Adjective'), 14],
 [('좋고', 'Adjective'), 12],
 [('만족합니다', 'Adjective'), 12],
 [('좋은', 'Adjective'), 12],
 [('이용', 'Noun'), 11],
 [('많이', 'Adverb'), 11],
 [('아직', 'Adverb'), 11],
 [('제품', 'Noun'), 10],
 [('좋네요', 'Adjective'), 10],
 [('해동', 'Noun'), 10],
 [('있어요', 'Adjective'), 10],
 [('냉동', 'Noun'), 9],
 [('안나', 'Noun'), 9],
 [('맛있고', 'Adjective'), 9],
 [('상태', 'Noun'), 9],
 [('빠른', 'Adjective'), 9],
 [('상품', 'Noun'), 8],
 [('같아요', 'Adjective'), 8],
 [('추천', 'Noun'), 8],
 [('아이스팩', 'Noun'), 7],
 [('최고', 'Noun'), 7],
 [('대비', 'Noun'), 6],
 [('저렴한', 'Adjective'), 6],
 [('저렴하게', '

In [32]:
summ, std = 0, 0
for word in result:
    summ += word[1]
avg = summ / len(result)
for word in result:
    std += abs(word[1]-avg)**2
std = std ** 0.5
std

90.363606586661

In [33]:
object_list = []
for word in result:
    tmp = {}
    tmp["text"]=word[0][0]
    tmp["value"]=round((math.log(word[1]) * 20), 3)
    object_list.append(tmp)
object_list

[{'text': '배송', 'value': 80.507},
 {'text': '좋아요', 'value': 77.003},
 {'text': '포장', 'value': 75.224},
 {'text': '맛있어요', 'value': 68.024},
 {'text': '구매', 'value': 65.917},
 {'text': '가격', 'value': 65.162},
 {'text': '목살', 'value': 64.378},
 {'text': '좋습니다', 'value': 64.378},
 {'text': '냄새', 'value': 63.561},
 {'text': '입니다', 'value': 60.89},
 {'text': '빠르고', 'value': 60.89},
 {'text': '주문', 'value': 60.89},
 {'text': '항상', 'value': 59.915},
 {'text': '가성', 'value': 54.161},
 {'text': '조아요', 'value': 54.161},
 {'text': '아주', 'value': 52.781},
 {'text': '맛있게', 'value': 52.781},
 {'text': '좋고', 'value': 49.698},
 {'text': '만족합니다', 'value': 49.698},
 {'text': '좋은', 'value': 49.698},
 {'text': '이용', 'value': 47.958},
 {'text': '많이', 'value': 47.958},
 {'text': '아직', 'value': 47.958},
 {'text': '제품', 'value': 46.052},
 {'text': '좋네요', 'value': 46.052},
 {'text': '해동', 'value': 46.052},
 {'text': '있어요', 'value': 46.052},
 {'text': '냉동', 'value': 43.944},
 {'text': '안나', 'value': 43.944},
 {'

In [61]:
def picker(pos, pos_tagged):
    result = []
    for sentence in pos_tagged:
        word_list = []
        for word in sentence:
            if word[1] == pos:
                word_list.append(word)
        result.append(word_list)
    return result